In [1]:
from transformers import (
    AutoTokenizer,
    LEDForConditionalGeneration,
)
from datasets import load_dataset, load_metric
import torch


from transformers import (
    AutoTokenizer,
    LEDConfig,
    LEDForConditionalGeneration,

)

/Users/sumersao/miniconda3/envs/twitbits/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset=load_dataset('multi_news')

Found cached dataset multi_news (/Users/sumersao/.cache/huggingface/datasets/multi_news/default/1.0.0/2f1f69a2bedc8ad1c5d8ae5148e4755ee7095f465c1c01ae8f85454342065a72)
100%|██████████| 3/3 [00:00<00:00, 107.74it/s]


In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_small = AutoTokenizer.from_pretrained("TinaLiHF/fined-tuned-T5small")
model_small = AutoModelForSeq2SeqLM.from_pretrained("TinaLiHF/fined-tuned-T5small")

pad_token_id = tokenizer_small.pad_token_id
docsep_token_id = tokenizer_small.convert_tokens_to_ids("<doc-sep>")

In [ ]:

def process_document_small(documents):
    input_ids_all=[]
    for data in documents:
        all_docs = data.split("|||||")[:-1]
        for i, doc in enumerate(all_docs):
            doc = doc.replace("\n", " ")
            doc = " ".join(doc.split())
            all_docs[i] = doc

        #### concat with global attention on doc-sep
        input_ids = []
        for doc in all_docs:
            input_ids.extend(
                tokenizer_small.encode(
                    doc,
                    truncation=True,
                    max_length=4096 // len(all_docs),
                )[1:-1]
            )
            input_ids.append(docsep_token_id)
        input_ids = (
            [tokenizer_small.bos_token_id]
            + input_ids
            + [tokenizer_small.eos_token_id]
        )
        input_ids_all.append(torch.tensor(input_ids))
    print(input_ids)
    # input_ids = torch.nn.utils.rnn.pad_sequence(
    #     input_ids_all, batch_first=True, padding_value=pad_token_id
    # )
    return input_ids


def batch_process_small(batch):
    input_ids=process_document_small(batch['document'])
    print(input_ids.shape)
    print(input_ids)
    # # get the input ids and attention masks together
    # global_attention_mask = torch.zeros_like(input_ids).to(input_ids.device)
    # # put global attention on <s> token

    # global_attention_mask[:, 0] = 1
    # global_attention_mask[input_ids == docsep_token_id] = 1

    # generated_ids = model_small.generate(
    #     input_ids=input_ids,
    #     attention_mask=global_attention_mask,
    #     use_cache=True,
    #     max_length=1024,
    #     num_beams=5,
    # )
    # generated_str = tokenizer_small.batch_decode(
    #         generated_ids.tolist(), skip_special_tokens=True
    #     )
    # result={}
    # result['generated_summaries'] = generated_str
    # result['gt_summaries']=batch['summary']
    result = {}
    return result

In [ ]:
import random
data_idx = random.choices(range(len(dataset['test'])),k=10)
dataset_small = dataset['test'].select(data_idx)
result_small = dataset_small.map(batch_process_small, batched=True, batch_size=2)